In [31]:
import pandas as pd
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import nltk
import string
from nltk.corpus import stopwords
from nlp_utils import *
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sn

In [32]:
#BART_classifier = transformers.pipeline('zero-shot-classification', model='facebook/bart-large-mnli')
#mDeBERTa_classifier = transformers.pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
#tokenizer = AutoTokenizer.from_pretrained("AyoubChLin/Bart-MNLI-CNN_news")
'''
model = AutoModelForSequenceClassification.from_pretrained("AyoubChLin/Bart-MNLI-CNN_news")
classifier = transformers.pipeline(
    "zero-shot-classification",
    model=model,
    tokenizer=tokenizer,
    device='cpu'
)'''
#classifier = transformers.pipeline("zero-shot-classification", 
#                       model="vicgalle/xlm-roberta-large-xnli-anli")

classifier = transformers.pipeline(
  'zero-shot-classification',
  model='sjrhuschlee/flan-t5-base-mnli',
  trust_remote_code=True,
)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at sjrhuschlee/flan-t5-base-mnli and are newly initialized: ['transformer.decoder.embed_tokens.weight', 'transformer.encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
df=pd.read_csv('datasets/US_stocks.csv')
labels = df['Industry Group'].unique()
sentences = df['Description'].head(n=500)
df=df.head(n=1000)

In [34]:
def tokenizer(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    stop_words = stopwords.words('english')
    words = [word for word in tokens if word not in string.punctuation]# and word not in stop_words]
    sentence=' '.join(words)
    return sentence

sentences=[tokenizer(sentence) for sentence in sentences]


In [35]:
#prediction = [BART_classifier(sentence, labels, multi_label=False) for sentence in sentences]
#prediction = [mDeBERTa_classifier(sentence, labels, multi_label=True) for sentence in sentences]
prediction = [classifier(sentence, labels, multi_label=True) for sentence in sentences]

In [ ]:
predicted_label=[prediction[i]['labels'][0] for i in range(df.shape[0])]
df['Predicted Label'] = predicted_label


In [ ]:
accuracy = accuracy_score(df['Industry Group'], df['Predicted Label'])
precision = precision_score(df['Industry Group'], df['Predicted Label'],average='weighted')
recall = recall_score(df['Industry Group'], df['Predicted Label'],average='weighted')
f1 = f1_score(df['Industry Group'], df['Predicted Label'],average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.406
Precision: 0.6643839909690714
Recall: 0.406
F1 Score: 0.43244357911950826


C:\Users\hugor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hugor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
